<a href="https://colab.research.google.com/github/youngbin03/Tiny-NeRF/blob/main/Fire%20extinguisher-3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NeRF-WandB-COLAB

### [NeRF 공식 저장소](https://github.com/bmild/nerf)를 수정하여 [wandb 연동이 편리하도록 만든 저장소](https://github.com/ProtossDragoon/nerf-wandb.git)의 소스코드를 이용해 커스텀 데이터셋에 NeRF 모델 학습

Plank-ing Hyundong 3D Reconstruction Project
Created 2022.07.12 <br>

**NoteBook Author** <br>
[Janghoo Lee](https://www.linkedin.com/in/janghoo-lee-25212a1a0) <br>
🎓 : [ProtossDragoon](https://github.com/ProtossDragoon) <br>
📧 : dlwkdgn3@gmail.com <br>

🤔 If you have any questions, please raise an issue in our [github repo](https://github.com/ProtossDragoon/PlankHyundong).

## 환경

### GPU 확인

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Mon Mar 11 00:07:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.1 MB/s eta 0:00:00


### TF 런타임 변경

In [ ]:
try:
    %tensorflow_version 2.x
except ValueError:
    # 만약 %tensorflow_version 1.x magic 명령어가 작동하지 않는 경우
    !pip uninstall --yes tensorflow
    !pip install tensorflow==1.15
    import tensorflow
    print(tensorflow.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


### 의존 패키지 설치

In [ ]:
!apt-get update
!sudo apt -qq install imagemagick
!pip install ConfigArgParse -qqq
!pip install imageio-ffmpeg -qqq

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.3).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


### NeRF 소스코드 다운로드

In [ ]:
!git clone https://github.com/ProtossDragoon/nerf-wandb.git

fatal: destination path 'nerf-wandb' already exists and is not an empty directory.


In [ ]:
%cd nerf-wandb
#!git checkout dev # 개발 중
!ls -al

/content/nerf-wandb
total 4360
drwxr-xr-x 8 root root    4096 Mar 11 00:01 .
drwxr-xr-x 1 root root    4096 Mar 10 23:57 ..
drwxr-xr-x 3 root root    4096 Mar 11 00:01 artifacts
-rw-r--r-- 1 root root     233 Mar 10 23:56 config_deepvoxels_greek.txt
-rw-r--r-- 1 root root     209 Mar 10 23:56 config_fern.txt
-rw-r--r-- 1 root root     221 Mar 10 23:56 config_lego.txt
-rw-r--r-- 1 root root     233 Mar 10 23:56 download_example_data.sh
-rw-r--r-- 1 root root     270 Mar 10 23:56 download_example_weights.sh
-rw-r--r-- 1 root root     279 Mar 10 23:56 environment.yml
-rw-r--r-- 1 root root 1253115 Mar 10 23:56 extract_mesh.ipynb
drwxr-xr-x 8 root root    4096 Mar 10 23:56 .git
-rw-r--r-- 1 root root     109 Mar 10 23:56 .gitignore
drwxr-xr-x 2 root root    4096 Mar 10 23:56 imgs
-rw-r--r-- 1 root root    1062 Mar 10 23:56 LICENSE
-rw-r--r-- 1 root root    2396 Mar 10 23:56 load_blender.py
-rw-r--r-- 1 root root    3890 Mar 10 23:56 load_deepvoxels.py
-rw-r--r-- 1 root root   10034 Mar 10 

### Weight and Bias 설치

- 실험 관리 도구로 wandb 를 사용합니다.
- 개인 wandb 프로젝트가 아닌, [plank-hyundong](https://wandb.ai/plank-hyundong) 팀의 [wandb 프로젝트 plank-hyundong](https://wandb.ai/plank-hyundong/plank-hyundong) 저장소를 사용하려면 관리자(dlwkdgn1@naver.com)에게 접근 키를 문의하세요.
- 개인 저장소를 사용하려면 `wandb.login()` 셀을 실행할 때 개인 계정으로 로그인하고, `python run_nerf.py` 셀을 실행할 때 파라미터 중 `--wandbproject` 와 `--wandbentity` 값을 변경하면 됩니다.

In [ ]:
!pip install wandb -qqq

In [ ]:
!pip install wandb

In [ ]:
# Log in to your W&B account
import wandb
wandb.login()

wandb: Currently logged in as: youngbi2021 (youngbi). Use `wandb login --relogin` to force relogin


True

In [ ]:
wandb.init()

wandb: Currently logged in as: youngbi2021 (youngbi). Use `wandb login --relogin` to force relogin


## 커스텀 데이터로 실행

- LLFF 를 통해 얻은 pose 가 필요합니다.
- LLFF 을 실행하려면 [Plankhyundong 저장소](https://github.com/ProtossDragoon/PlankHyundong)의 [colmap_colab 노트북](https://github.com/ProtossDragoon/PlankHyundong/blob/main/notebooks/colmap_colab.ipynb)을 참고하세요.

### 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


### 파라미터 설정

In [ ]:
from datetime import datetime
now = datetime.now().strftime('%y%m%d_%H%M%S')

dataset_name = 'hyundong360_50' #@param ['hyundong360_50', 'hyundong360_100', 'hyundong360_200', 'hyundong360_400', 'noparking360_100', 'noparking360_200', 'noparking360_400', 'noparking720_100', 'noparking720_200', 'noparking720_400', 'hyundong360_100_background', 'hyundong360_100_cleanup', 'hyundong360_50_cleanup', 'hyundong360_25_cleanup']
downsample_factor = 4 #@param {type:"slider", min:1, max:64, step:1}
netdepth = 6 #@param {type:"slider", min:4, max:16, step:2}
netwidth = 96 #@param {type:"slider", min:64, max:256, step:4}
experiment_name = f'{dataset_name}_{downsample_factor}_downsampled_{now}'
max_iter = 30000 #@param
learning_rate = 0.001 #@param
video_saving_cnt = 3 #@param {type:"slider", min:1, max:10, step:1}
n_samples = 128 #@param {type:"slider", min:32, max:256, step:32}

# fine 모델에서 사용되는 샘플 개수는 coarse 모델의 sampling 개수의 2배로 설정한다.
# 공식 논문에서 제안하는 대로, 64이면 128.
n_importance = n_samples * 2

# Reproduce 를 위해 고정 random_seed 를 사용
random_seed = 777 #@param

# tradeoff: memory <-> speed (training 에는 속도와 성능 모두에 영향을 미치지 않음. 학습 도중 동영상을 만들 때 OOM 이 난다면 충분히 낮출 것)
rendering_speed = 2048 #@param {type:"slider", min:1024, max:16384, step:1024}

# tradeoff: memory <-> result
n_points_per_ray = 65536 #@param {type:"slider", min:2048, max:262144, step:1024}

_dummy_dir = '/content/drive/MyDrive/fire/data/logs/{experiment_name}'
_tensorboard_logdir = '/content/drive/MyDrive/fire/data/logs/summaries/{experiment_name}'
print(f'experiment: {experiment_name}')

experiment: hyundong360_50_4_downsampled_240311_000757


Mesh 코드를 실행하기 위한 config.txt 생성

In [ ]:
# Make config.txt file.
f = open("/content/drive/MyDrive/fire/data/logs/config.txt", 'w')
def make_config(a, b):

  if b is None:
    f.write(a + '\n')
  else:
    data = a + ' = ' + b + '\n'
    f.write(data)


make_config("maxiter", str(max_iter))
make_config("datadir", '/content/drive/MyDrive/fire/data')
make_config("basedir", '/content/drive/MyDrive/fire/data/logs')
make_config("dataset_type", 'llff')
make_config('factor', str(downsample_factor))
make_config('netdepth', str(netdepth))
make_config('netwidth', str(netwidth))
make_config('netdepth_fine', str(netdepth))
make_config('netwidth_fine', str(netwidth))
make_config('chunk', str(rendering_speed))
make_config('netchunk', str(n_points_per_ray))
make_config('lrate', str(learning_rate))
make_config('i_video', str(max_iter // video_saving_cnt))
make_config('expname', experiment_name)
make_config('N_samples', str(n_samples))
make_config('N_importance', str(n_importance))
make_config('random_seed', str(random_seed))
make_config('raw_noise_std', '1.0')
make_config('use_viewdirs', None)
make_config('no_ndc', None)
make_config('spherify', None)
make_config('lindisp', None)

f.close()

### 학습 시작

- 360도에 걸쳐 촬영한 데이터의 경우, `--no_ndc`, `--spherify`, `--lindisp` 를 추가하세요.

In [ ]:
!python run_nerf.py \
    --wandbproject {'fire-youngbi'} \
    --wandbentity {'youngbi2021'} \
    --maxiter {max_iter} \
    --datadir /content/drive/MyDrive/fire/data \
    --basedir /content/drive/MyDrive/fire/data/logs \
    --dataset_type llff \
    --factor {downsample_factor} \
    --netdepth {netdepth} \
    --netwidth {netwidth} \
    --netdepth_fine {netdepth} \
    --netwidth_fine {netwidth} \
    --chunk {rendering_speed} \
    --netchunk {n_points_per_ray} \
    --lrate {learning_rate} \
    --i_video {max_iter // video_saving_cnt} \
    --expname {experiment_name} \
    --N_samples {n_samples} \
    --N_importance {n_importance} \
    --random_seed {random_seed} \
    --raw_noise_std 1.0 \
    --use_viewdirs \
    --no_ndc \
    --spherify \
    --lindisp

2024-03-11 00:43:50.332746: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 00:43:50.332793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 00:43:50.334190: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 00:43:50.341579: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 00:43:51.332630: W tensorflow/comp